In [0]:
import warnings
import numpy as np
import pandas as pd
import xgboost as xg
import numpy as np
import math
import scipy.stats as stats
from math import sqrt
from datetime import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [0]:
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
schema = StructType([StructField("ItemID",StringType(),True),\
StructField("Productline",StringType(),True),\
StructField("SeriesName",StringType(),True),\
StructField("SeriesNumber",StringType(),True),\
StructField("Status",StringType(),True),\
StructField("FiscalYear",IntegerType(),True),\
StructField("FiscalWeek",IntegerType(),True),\
StructField("AnniversarySale_MainEvent",IntegerType(),True),\
StructField("PostIndependenceDay_PostTentpole",IntegerType(),True),\
StructField("PreBlackFriday_PreTentpole",IntegerType(),True),\
StructField("PreIndependenceDay_PreTentpole",IntegerType(),True),\
StructField("PreNewYear_PreTentpole",IntegerType(),True),\
StructField("CyberMonday_MainEvent",IntegerType(),True),\
StructField("PostBlackFriday_PostTentpole",IntegerType(),True),\
StructField("PostMemorialDay_PostTentpole",IntegerType(),True),\
StructField("MemorialDay_MainEvent",IntegerType(),True),\
StructField("PreLaborDay_PostTentpole",IntegerType(),True),\
StructField("PreAnniversarySale_PreTentpole",IntegerType(),True),\
StructField("BlackFriday_PostTentpole",IntegerType(),True),\
StructField("PostAnniversarySale_PostTentpole",IntegerType(),True),\
StructField("PostLaborDay_PostTentpole",IntegerType(),True),\
StructField("PostNewYear_PostTentpole",IntegerType(),True),\
StructField("LaborDay_MainEvent",IntegerType(),True),\
StructField("BlackFriday_MainEvent",IntegerType(),True),\
StructField("PostPresidentsDay_PostTentpole",IntegerType(),True),\
StructField("PreLaborDay_PreTentpole",IntegerType(),True),\
StructField("IndependenceDay_MainEvent",IntegerType(),True),\
StructField("PreMemorialDay_PreTentpole",IntegerType(),True),\
StructField("NewYear_MainEvent",IntegerType(),True),\
StructField("PrePresidentsDay_PreTentpole",IntegerType(),True),\
StructField("PresidentsDay_MainEvent",IntegerType(),True),\
StructField("January",IntegerType(),True),\
StructField("Feburary",IntegerType(),True),\
StructField("March",IntegerType(),True),\
StructField("April",IntegerType(),True),\
StructField("May",IntegerType(),True),\
StructField("June",IntegerType(),True),\
StructField("July",IntegerType(),True),\
StructField("August",IntegerType(),True),\
StructField("September",IntegerType(),True),\
StructField("October",IntegerType(),True),\
StructField("November",IntegerType(),True),\
StructField("December",IntegerType(),True),\
StructField("AFHSPGZ1",IntegerType(),True),\
StructField("AFHSPGZ2",IntegerType(),True),\
StructField("AFHSPGZ3",IntegerType(),True),\
StructField("AFHSPGZ5",IntegerType(),True),\
StructField("AFHSPGZ6",IntegerType(),True),\
StructField("AFHSPGZ7",IntegerType(),True),\
StructField("AFHSPGZ8",IntegerType(),True),\
StructField("SALES",FloatType(),True),\
StructField("Quantity",FloatType(),True),\
StructField("Discounts",FloatType(),True),\
StructField("COGS",FloatType(),True),\
StructField("PrevWeekSales",FloatType(),True),\
StructField("PrevWeek2Sales",FloatType(),True),\
StructField("PrevWeek3Sales",FloatType(),True)
])

In [0]:
csvdf = spark.read.format("csv").option("header",True).schema(schema).load("/mnt/refined-zone/Distribution/Test_Analytics/")

In [0]:
df = csvdf.toPandas()

In [0]:
df.shape

Out[128]: (8097705, 57)

In [0]:
data = df[df['Quantity']>0]

In [0]:
data.shape

Out[166]: (1251037, 57)

In [0]:
round((1251037/8097705)*100,1)

Out[167]: 15.4

In [0]:
tentpole_cols = list(df.iloc[:,7:-26].columns)

In [0]:
data["Holiday"] = data[tentpole_cols].apply(lambda x: 1 if sum(x)>0 else 0, axis=1)

In [0]:
data.rename(columns={'SALES':'Sales'}, inplace=True)

In [0]:
# Creating new features
data['AverageWeeklyPricePreCoupon'] = data['Sales'] / data['Quantity']
data['AverageWeeklyPrice'] = (data['Sales'] - data['Discounts']) / data['Quantity']
data['AverageWeeklyCOGS'] = data['COGS'] / data['Quantity']
data['AverageWeeklyPricePreCoupon'] = data['AverageWeeklyPricePreCoupon'].replace(0, np.nan)
data['AverageWeeklyCOGS'] = data['AverageWeeklyCOGS'].replace(0, np.nan)
data['AverageWeeklyPrice'] = data['AverageWeeklyPrice'].replace(0, np.nan)  # replace nulll values with 0
data.dropna(subset=['AverageWeeklyPrice', 'AverageWeeklyCOGS'], inplace=True)

In [0]:
data.head(3)

,ItemID,Productline,SeriesName,SeriesNumber,Status,FiscalYear,FiscalWeek,AnniversarySale_MainEvent,PostIndependenceDay_PostTentpole,PreBlackFriday_PreTentpole,PreIndependenceDay_PreTentpole,PreNewYear_PreTentpole,CyberMonday_MainEvent,PostBlackFriday_PostTentpole,PostMemorialDay_PostTentpole,MemorialDay_MainEvent,PreLaborDay_PostTentpole,PreAnniversarySale_PreTentpole,BlackFriday_PostTentpole,PostAnniversarySale_PostTentpole,PostLaborDay_PostTentpole,PostNewYear_PostTentpole,LaborDay_MainEvent,BlackFriday_MainEvent,PostPresidentsDay_PostTentpole,PreLaborDay_PreTentpole,IndependenceDay_MainEvent,PreMemorialDay_PreTentpole,NewYear_MainEvent,PrePresidentsDay_PreTentpole,PresidentsDay_MainEvent,January,Feburary,March,April,May,June,July,August,September,October,November,December,AFHSPGZ1,AFHSPGZ2,AFHSPGZ3,AFHSPGZ5,AFHSPGZ6,AFHSPGZ7,AFHSPGZ8,Sales,Quantity,Discounts,COGS,PrevWeekSales,PrevWeek2Sales,PrevWeek3Sales,Holiday,AverageWeeklyPricePreCoupon,AverageWeeklyPrice,AverageWeeklyCOGS
18,4130316,Stationary,Eltmann,41303,Current,2019,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.00,214.250000,0.0,0.0,0.0,0,400.0,440.00,214.250000
21,4130316,Stationary,Eltmann,41303,Current,2019,22,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.00,214.250000,0.0,0.0,400.0,1,400.0,440.00,214.250000
41,4870155,Stationary,Olsberg,48701,Current,2019,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,487.5,1.0,-48.75,237.649994,0.0,0.0,0.0,0,487.5,536.25,237.649994


In [0]:
## creating z scores
data['PriceZone'] = data[['AFHSPGZ1', 'AFHSPGZ2', 'AFHSPGZ3', 'AFHSPGZ5','AFHSPGZ6', 'AFHSPGZ7', 'AFHSPGZ8']].idxmax(axis=1)
new_data = data.groupby(['ItemID','PriceZone'])['AverageWeeklyPrice'].mean().reset_index()
new_data.rename(columns={'AverageWeeklyPrice':'Avg_Price'}, inplace=True)
data =data.merge(new_data,on =['ItemID','PriceZone'])
#new_data.head()

In [0]:
data.head()

,ItemID,Productline,SeriesName,SeriesNumber,Status,FiscalYear,FiscalWeek,AnniversarySale_MainEvent,PostIndependenceDay_PostTentpole,PreBlackFriday_PreTentpole,PreIndependenceDay_PreTentpole,PreNewYear_PreTentpole,CyberMonday_MainEvent,PostBlackFriday_PostTentpole,PostMemorialDay_PostTentpole,MemorialDay_MainEvent,PreLaborDay_PostTentpole,PreAnniversarySale_PreTentpole,BlackFriday_PostTentpole,PostAnniversarySale_PostTentpole,PostLaborDay_PostTentpole,PostNewYear_PostTentpole,LaborDay_MainEvent,BlackFriday_MainEvent,PostPresidentsDay_PostTentpole,PreLaborDay_PreTentpole,IndependenceDay_MainEvent,PreMemorialDay_PreTentpole,NewYear_MainEvent,PrePresidentsDay_PreTentpole,PresidentsDay_MainEvent,January,Feburary,March,April,May,June,July,August,September,October,November,December,AFHSPGZ1,AFHSPGZ2,AFHSPGZ3,AFHSPGZ5,AFHSPGZ6,AFHSPGZ7,AFHSPGZ8,Sales,Quantity,Discounts,COGS,PrevWeekSales,PrevWeek2Sales,PrevWeek3Sales,Holiday,AverageWeeklyPricePreCoupon,AverageWeeklyPrice,AverageWeeklyCOGS,PriceZone,Avg_Price
0,4130316,Stationary,Eltmann,41303,Current,2019,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.0,214.250000,0.0,0.0,0.0,0,400.0,440.0,214.250000,AFHSPGZ8,593.168518
1,4130316,Stationary,Eltmann,41303,Current,2019,22,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.0,214.250000,0.0,0.0,400.0,1,400.0,440.0,214.250000,AFHSPGZ8,593.168518
2,4130316,Stationary,Eltmann,41303,Current,2019,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,325.0,1.0,-32.5,226.050003,0.0,0.0,0.0,0,325.0,357.5,226.050003,AFHSPGZ8,593.168518
3,4130316,Stationary,Eltmann,41303,Current,2020,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,425.0,1.0,0.0,215.850006,0.0,0.0,0.0,0,425.0,425.0,215.850006,AFHSPGZ8,593.168518
4,4130316,Stationary,Eltmann,41303,Current,2020,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,425.0,1.0,0.0,215.850006,425.0,0.0,0.0,1,425.0,425.0,215.850006,AFHSPGZ8,593.168518


In [0]:
new_data.head()

,ItemID,PriceZone,Avg_Price
0,1010425,AFHSPGZ1,409.529297
1,1010425,AFHSPGZ2,439.229156
2,1010425,AFHSPGZ3,413.019684
3,1010425,AFHSPGZ5,403.190002
4,1010425,AFHSPGZ6,399.119080


In [0]:

#data.drop(['Avg_Price'],inplace= True)
data['Diff_from_avg'] = data['AverageWeeklyPrice'] - data['Avg_Price']
data['log_value'] = (np.log(1 + data['AverageWeeklyPrice'] / data['Avg_Price']))

data['zscore1'] = stats.zscore(data['Diff_from_avg'])
data['zscore2'] = stats.zscore(data['log_value'])

data.head()


,ItemID,Productline,SeriesName,SeriesNumber,Status,FiscalYear,FiscalWeek,AnniversarySale_MainEvent,PostIndependenceDay_PostTentpole,PreBlackFriday_PreTentpole,PreIndependenceDay_PreTentpole,PreNewYear_PreTentpole,CyberMonday_MainEvent,PostBlackFriday_PostTentpole,PostMemorialDay_PostTentpole,MemorialDay_MainEvent,PreLaborDay_PostTentpole,PreAnniversarySale_PreTentpole,BlackFriday_PostTentpole,PostAnniversarySale_PostTentpole,PostLaborDay_PostTentpole,PostNewYear_PostTentpole,LaborDay_MainEvent,BlackFriday_MainEvent,PostPresidentsDay_PostTentpole,PreLaborDay_PreTentpole,IndependenceDay_MainEvent,PreMemorialDay_PreTentpole,NewYear_MainEvent,PrePresidentsDay_PreTentpole,PresidentsDay_MainEvent,January,Feburary,March,April,May,June,July,August,September,October,November,December,AFHSPGZ1,AFHSPGZ2,AFHSPGZ3,AFHSPGZ5,AFHSPGZ6,AFHSPGZ7,AFHSPGZ8,Sales,Quantity,Discounts,COGS,PrevWeekSales,PrevWeek2Sales,PrevWeek3Sales,Holiday,AverageWeeklyPricePreCoupon,AverageWeeklyPrice,AverageWeeklyCOGS,PriceZone,Avg_Price,Diff_from_avg,log_value,zscore1,zscore2
0,4130316,Stationary,Eltmann,41303,Current,2019,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.0,214.250000,0.0,0.0,0.0,0,400.0,440.0,214.250000,AFHSPGZ8,593.168518,-153.168518,0.554907,-2.151164,-1.568928
1,4130316,Stationary,Eltmann,41303,Current,2019,22,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,400.0,1.0,-40.0,214.250000,0.0,0.0,400.0,1,400.0,440.0,214.250000,AFHSPGZ8,593.168518,-153.168518,0.554907,-2.151164,-1.568928
2,4130316,Stationary,Eltmann,41303,Current,2019,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,325.0,1.0,-32.5,226.050003,0.0,0.0,0.0,0,325.0,357.5,226.050003,AFHSPGZ8,593.168518,-235.668518,0.471687,-3.309829,-2.539738
3,4130316,Stationary,Eltmann,41303,Current,2020,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,425.0,1.0,0.0,215.850006,0.0,0.0,0.0,0,425.0,425.0,215.850006,AFHSPGZ8,593.168518,-168.168518,0.540282,-2.361830,-1.739535
4,4130316,Stationary,Eltmann,41303,Current,2020,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,425.0,1.0,0.0,215.850006,425.0,0.0,0.0,1,425.0,425.0,215.850006,AFHSPGZ8,593.168518,-168.168518,0.540282,-2.361830,-1.739535


In [0]:
data['Avg_Price']

Out[177]: 0 593.168518
1 593.168518
2 593.168518
3 593.168518
4 593.168518
 ... 
1249825 422.209991
1249826 422.209991
1249827 422.209991
1249828 422.209991
1249829 422.209991
Name: Avg_Price, Length: 1249830, dtype: float32

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-46515125115184> in <module> 
 ----> 1 float ( math . log ( 1 + data [ 'AverageWeeklyPrice' ] / data [ 'Avg_Price' ] ) ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/series.py in wrapper (self) 
 110 if len ( self ) == 1 : 
 111 return converter ( self . iloc [ 0 ] ) 
 --> 112 raise TypeError ( f"cannot convert the series to {converter}" ) 
 113 
 114 wrapper . __name__ = f"__{converter.__name__}__" 

 TypeError : cannot convert the series to <class 'float'>

In [0]:
data.shape #1249740, 61)

Out[178]: (1249830, 67)

In [0]:
data.columns

Out[179]: Index(['ItemID', 'Productline', 'SeriesName', 'SeriesNumber', 'Status',
 'FiscalYear', 'FiscalWeek', 'AnniversarySale_MainEvent',
 'PostIndependenceDay_PostTentpole', 'PreBlackFriday_PreTentpole',
 'PreIndependenceDay_PreTentpole', 'PreNewYear_PreTentpole',
 'CyberMonday_MainEvent', 'PostBlackFriday_PostTentpole',
 'PostMemorialDay_PostTentpole', 'MemorialDay_MainEvent',
 'PreLaborDay_PostTentpole', 'PreAnniversarySale_PreTentpole',
 'BlackFriday_PostTentpole', 'PostAnniversarySale_PostTentpole',
 'PostLaborDay_PostTentpole', 'PostNewYear_PostTentpole',
 'LaborDay_MainEvent', 'BlackFriday_MainEvent',
 'PostPresidentsDay_PostTentpole', 'PreLaborDay_PreTentpole',
 'IndependenceDay_MainEvent', 'PreMemorialDay_PreTentpole',
 'NewYear_MainEvent', 'PrePresidentsDay_PreTentpole',
 'PresidentsDay_MainEvent', 'January', 'Feburary', 'March', 'April',
 'May', 'June', 'July', 'August', 'September', 'October', 'November',
 'December', 'AFHSPGZ1', 'AFHSPGZ2', 'AFHSPGZ3', 'AFHSPGZ5', 'AFHSPGZ6',
 'AFHSPGZ7', 'AFHSPGZ8', 'Sales', 'Quantity', 'Discounts', 'COGS',
 'PrevWeekSales', 'PrevWeek2Sales', 'PrevWeek3Sales', 'Holiday',
 'AverageWeeklyPricePreCoupon', 'AverageWeeklyPrice',
 'AverageWeeklyCOGS', 'PriceZone', 'Avg_Price', 'Diff_from_avg',
 'log_value', 'zscore1', 'zscore2'],
 dtype='object')

In [0]:
# Adding Other Variables

data['LogPrice'] = np.log(data['AverageWeeklyPrice'])

data['Holiday_Prev_week'] = data['PrevWeekSales'] * data['Holiday']
data['Holiday_Prev_week2'] = data['PrevWeek2Sales'] * data['Holiday']

data.dropna(subset=(['PrevWeekSales', 'PrevWeek2Sales', 'PrevWeek3Sales']), inplace=True)

In [0]:
data.columns

Out[186]: Index(['ItemID', 'Productline', 'SeriesName', 'SeriesNumber', 'Status',
 'FiscalYear', 'FiscalWeek', 'AnniversarySale_MainEvent',
 'PostIndependenceDay_PostTentpole', 'PreBlackFriday_PreTentpole',
 'PreIndependenceDay_PreTentpole', 'PreNewYear_PreTentpole',
 'CyberMonday_MainEvent', 'PostBlackFriday_PostTentpole',
 'PostMemorialDay_PostTentpole', 'MemorialDay_MainEvent',
 'PreLaborDay_PostTentpole', 'PreAnniversarySale_PreTentpole',
 'BlackFriday_PostTentpole', 'PostAnniversarySale_PostTentpole',
 'PostLaborDay_PostTentpole', 'PostNewYear_PostTentpole',
 'LaborDay_MainEvent', 'BlackFriday_MainEvent',
 'PostPresidentsDay_PostTentpole', 'PreLaborDay_PreTentpole',
 'IndependenceDay_MainEvent', 'PreMemorialDay_PreTentpole',
 'NewYear_MainEvent', 'PrePresidentsDay_PreTentpole',
 'PresidentsDay_MainEvent', 'January', 'Feburary', 'March', 'April',
 'May', 'June', 'July', 'August', 'September', 'October', 'November',
 'December', 'AFHSPGZ1', 'AFHSPGZ2', 'AFHSPGZ3', 'AFHSPGZ5', 'AFHSPGZ6',
 'AFHSPGZ7', 'AFHSPGZ8', 'Sales', 'Quantity', 'Discounts', 'COGS',
 'PrevWeekSales', 'PrevWeek2Sales', 'PrevWeek3Sales', 'Holiday',
 'AverageWeeklyPricePreCoupon', 'AverageWeeklyPrice',
 'AverageWeeklyCOGS', 'PriceZone', 'Avg_Price', 'Diff_from_avg',
 'log_value', 'zscore1', 'zscore2', 'LogPrice', 'Holiday_Prev_week',
 'Holiday_Prev_week2'],
 dtype='object')

In [0]:
data.dropna(inplace=True)

data = data.reset_index(drop=True)

# Data For Training
train_data = data.drop(data[(data['FiscalYear'] == 2022)&(data['FiscalWeek'] > 22)].index)

# Data For Testing
df_test = data[(data['FiscalYear'] == 2022) & (data['FiscalWeek'] == 23)].reset_index(drop = True)

In [0]:
df_test.shape

Out[183]: (7292, 70)

In [0]:
# Model Building

XGBR = xg.XGBRegressor(n_estimators=100, objectvie='reg:squarederror', booster='gbtree',random_state=42, learning_rate=0.05)

train_df = pd.DataFrame()
evaluation_dict = {"Product line":[], "R-Square":[], "RMSE":[], "MAE":[]}
models = dict()

prod_line_list = list(data['Productline'].unique())

remove_cols = ['ItemID', 'Productline','FiscalYear','Quantity', 'Discounts', 'Sales', 'COGS', 'SeriesName', 'SeriesNumber', 'Status','PriceZone', 'Avg_Price', 'Diff_from_avg','log_value',]

for prod in prod_line_list:
    temp = train_data[train_data.Productline == prod]
    
    X = temp.drop(remove_cols,axis=1)

    y = temp['Quantity']

    mod = XGBR.fit(X,y)
    models[prod] = mod
    temp['Predicted_Quantity'] = np.round(mod.predict(X),1)
    train_df = pd.concat([train_df,temp], axis=0, ignore_index=True)

# Evaluation
    evaluation_dict["Product line"].append(prod)
    evaluation_dict["R-Square"].append(round(sqrt(r2_score(temp['Quantity'], temp['Predicted_Quantity'])),2))
    evaluation_dict["RMSE"].append(round(sqrt(mean_squared_error(temp['Quantity'], temp['Predicted_Quantity'])),2))
    evaluation_dict["MAE"].append(round(mean_absolute_error(temp['Quantity'], temp['Predicted_Quantity']),2))

metric_df = pd.DataFrame(evaluation_dict)

[09:10:50] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:11:14] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:11:31] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:11:44] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:12:28] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:12:47] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:13:00] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:13:08] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:13:12] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.


[09:13:16] WARNING: /workspace/src/learner.cc:480: 
Parameters: { objectvie } might not be used.

 This may not be accurate due to some parameters are only used in language bindings but
 passed down to XGBoost core. Or some parameters are not used but slip through this
 verification. Please open an issue if you find above cases.

In [0]:
new_price = spark.read.format("csv").option("header",True).load("/mnt/refined-zone/Distribution/Test_Analytics1/")
nw_prc = new_price.toPandas()

In [0]:
nw_prc['Sku'] = [str(i).strip() for i in nw_prc['Sku']]
id_lst = list(df_test.ItemID.unique())

new_price_df = nw_prc[nw_prc['Sku'].isin(id_lst)][['Sku', 'Status','Z1Best', 'Z2Best','Z3Best', 'Z5Best', 'Z6Best', 'Z7Best','Z8Best']]
new_price_df.rename(columns={'Z1Best':'AFHSPGZ1', 'Z2Best':'AFHSPGZ2', 'Z3Best':'AFHSPGZ3', 'Z5Best':'AFHSPGZ5', 'Z6Best':'AFHSPGZ6',
       'Z7Best':'AFHSPGZ7', 'Z8Best':'AFHSPGZ8'}, inplace=True)

In [0]:
df_test['PriceZone'] = df_test[['AFHSPGZ1', 'AFHSPGZ2', 'AFHSPGZ3', 'AFHSPGZ5','AFHSPGZ6', 'AFHSPGZ7', 'AFHSPGZ8']].idxmax(axis=1)

In [0]:
sku_list = list(new_price_df.Sku.unique())
for i in range(len(df_test)):
  x,y = df_test.loc[i,'ItemID'], df_test.loc[i,'PriceZone']
  if x in sku_list:
    z = new_price_df[new_price_df['Sku']==x][y].iloc[0]
    df_test.loc[i,'AverageWeeklyPrice'] = float(z)

In [0]:
df_test.drop(['PriceZone'],axis=1,inplace=True)
df_test['LogPrice'] = np.log(df_test['AverageWeeklyPrice'])

In [0]:
print(new_price_df.shape)
print(df_test.shape)

(3253, 9)
(7292, 69)

In [0]:
# ddd = df_test.merge(new_price_df,left_on='ItemID', right_on='Sku', how='left')
# ddd[['ItemID','Sku','AverageWeeklyPrice','AFHSPGZ1_y','AFHSPGZ1_x','AFHSPGZ2_y','AFHSPGZ2_x','AFHSPGZ3_y','AFHSPGZ3_x','AFHSPGZ5_y','AFHSPGZ5_x','AFHSPGZ6_y','AFHSPGZ7_y','AFHSPGZ8_y']].head(10)

In [0]:
X.columns

Out[195]: Index(['FiscalWeek', 'AnniversarySale_MainEvent',
 'PostIndependenceDay_PostTentpole', 'PreBlackFriday_PreTentpole',
 'PreIndependenceDay_PreTentpole', 'PreNewYear_PreTentpole',
 'CyberMonday_MainEvent', 'PostBlackFriday_PostTentpole',
 'PostMemorialDay_PostTentpole', 'MemorialDay_MainEvent',
 'PreLaborDay_PostTentpole', 'PreAnniversarySale_PreTentpole',
 'BlackFriday_PostTentpole', 'PostAnniversarySale_PostTentpole',
 'PostLaborDay_PostTentpole', 'PostNewYear_PostTentpole',
 'LaborDay_MainEvent', 'BlackFriday_MainEvent',
 'PostPresidentsDay_PostTentpole', 'PreLaborDay_PreTentpole',
 'IndependenceDay_MainEvent', 'PreMemorialDay_PreTentpole',
 'NewYear_MainEvent', 'PrePresidentsDay_PreTentpole',
 'PresidentsDay_MainEvent', 'January', 'Feburary', 'March', 'April',
 'May', 'June', 'July', 'August', 'September', 'October', 'November',
 'December', 'AFHSPGZ1', 'AFHSPGZ2', 'AFHSPGZ3', 'AFHSPGZ5', 'AFHSPGZ6',
 'AFHSPGZ7', 'AFHSPGZ8', 'PrevWeekSales', 'PrevWeek2Sales',
 'PrevWeek3Sales', 'Holiday', 'AverageWeeklyPricePreCoupon',
 'AverageWeeklyPrice', 'AverageWeeklyCOGS', 'zscore1', 'zscore2',
 'LogPrice', 'Holiday_Prev_week', 'Holiday_Prev_week2'],
 dtype='object')

In [0]:
# prediction Test Data
forecast = pd.DataFrame()
for prod in prod_line_list:
  forecasted_df = df_test[df_test['Productline'] == prod].reset_index(drop = True)
  test_x = forecasted_df[list(X.columns)]
  forecasted_df['Predicted_Quantity'] = np.round(models[prod].predict(test_x),1)
  forecasted_df['Pred_Sales'] = forecasted_df['Predicted_Quantity'] * forecasted_df['AverageWeeklyPrice']
  forecast = pd.concat([forecast,forecasted_df], axis=0, ignore_index=True)
  
forecast.rename(columns={'AverageWeeklyPrice':'Price'}, inplace=True)

In [0]:
forecast.Sales.sum()

Out[197]: 8070431.0

In [0]:
forecast.Pred_Sales.sum()

Out[198]: 13067176.0

In [0]:
df_sp = spark.createDataFrame(forecast)
display(forecast)

ItemID,Productline,SeriesName,SeriesNumber,Status,FiscalYear,FiscalWeek,AnniversarySale_MainEvent,PostIndependenceDay_PostTentpole,PreBlackFriday_PreTentpole,PreIndependenceDay_PreTentpole,PreNewYear_PreTentpole,CyberMonday_MainEvent,PostBlackFriday_PostTentpole,PostMemorialDay_PostTentpole,MemorialDay_MainEvent,PreLaborDay_PostTentpole,PreAnniversarySale_PreTentpole,BlackFriday_PostTentpole,PostAnniversarySale_PostTentpole,PostLaborDay_PostTentpole,PostNewYear_PostTentpole,LaborDay_MainEvent,BlackFriday_MainEvent,PostPresidentsDay_PostTentpole,PreLaborDay_PreTentpole,IndependenceDay_MainEvent,PreMemorialDay_PreTentpole,NewYear_MainEvent,PrePresidentsDay_PreTentpole,PresidentsDay_MainEvent,January,Feburary,March,April,May,June,July,August,September,October,November,December,AFHSPGZ1,AFHSPGZ2,AFHSPGZ3,AFHSPGZ5,AFHSPGZ6,AFHSPGZ7,AFHSPGZ8,Sales,Quantity,Discounts,COGS,PrevWeekSales,PrevWeek2Sales,PrevWeek3Sales,Holiday,AverageWeeklyPricePreCoupon,Price,AverageWeeklyCOGS,Avg_Price,Diff_from_avg,log_value,zscore1,zscore2,LogPrice,Holiday_Prev_week,Holiday_Prev_week2,Predicted_Quantity,Pred_Sales
4130316,Stationary,Eltmann,41303,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1829.969970703125,3.0,-121.11000061035156,790.02001953125,1219.97998046875,1219.97998046875,0.0,1,609.989990234375,759.989990234375,263.3399963378906,567.4258422851562,82.93414306640625,0.7636796236038208,1.1647623777389526,0.8665190935134888,6.633305072784424,1219.97998046875,1219.97998046875,3.799999952316284,2887.9619140625
1240434,Stationary,Carnaby,12404,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,550.0,1.0,0.0,228.0,0.0,550.0,550.0,1,550.0,680.0,228.0,557.3333129882812,-7.33331298828125,0.6865465044975281,-0.10299216955900192,-0.03328155353665352,6.522092819213867,0.0,550.0,1.899999976158142,1292.0
5960308,Stationary,Lavernett,59603,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,489.989990234375,1.0,-49.0,211.86000061035156,0.0,0.0,0.0,1,489.989990234375,519.989990234375,211.86000061035156,538.989990234375,0.0,0.6931471824645996,1.7009234298814846E-10,0.04371901974081993,6.253809452056885,0.0,0.0,1.2000000476837158,623.988037109375
1202135,Stationary,Bladen,12021,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2799.929931640625,7.0,-242.55999755859375,1720.1199951171875,1199.969970703125,4799.8798828125,5599.85986328125,1,399.989990234375,399.989990234375,245.73143005371094,439.34661865234375,-4.7052001953125,0.6877780556678772,-0.06608182936906815,-0.018914824351668358,5.991439342498779,1199.969970703125,4799.8798828125,7.699999809265137,3079.9228515625
1240434,Stationary,Carnaby,12404,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,10800.0,20.0,-407.6499938964844,4506.919921875,9720.0,5940.0,8900.0,1,540.0,670.0,225.3459930419922,556.692626953125,3.68988037109375,0.6964558362960815,0.051822248846292496,0.08231630176305771,6.507277488708496,9720.0,5940.0,18.799999237060547,12595.9990234375
1330266,Stationary,Castano,13302,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1939.97998046875,2.0,-145.5,772.1599731445312,6789.93017578125,7399.91015625,2339.969970703125,1,969.989990234375,1069.989990234375,386.0799865722656,911.7572631835938,130.98272705078125,0.7625144720077515,1.839577078819275,0.8529269695281982,6.975404739379883,6789.93017578125,7399.91015625,9.399999618530273,10057.9052734375
5960308,Stationary,Lavernett,59603,Current,2022,23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1919.9599609375,4.0,-187.1999969482422,856.02001953125,479.989990234375,1799.9599609375,449.989990234375,1,479.989990234375,519.989990234375,214.0050048828125,508.4566650390625,18.33331298828125,0.71101498603820

In [0]:
len(data.ItemID.unique())

Out[200]: 5527

In [0]:
len(train_df.ItemID.unique())

Out[201]: 5503

In [0]:
len(forecast.ItemID.unique())

Out[202]: 3253

In [0]:
metric_df

,Product line,R-Square,RMSE,MAE
0,Stationary,0.91,2.63,1.34
1,Accessories,0.83,1.78,1.04
2,Motion,0.86,1.38,0.84
3,Bedroom,0.93,3.21,1.60
4,Dining,0.94,8.51,4.09
5,Occasional,0.91,2.36,1.40
6,Bedding,0.98,6.08,2.87
7,Entertainment,0.93,2.29,1.37
8,Outdoor,0.91,2.49,1.57
9,Home Office,0.91,2.42,1.50


In [0]:
forecast.Quantity.sum()

Out[204]: 23461.998

In [0]:
forecast.Predicted_Quantity.sum()

Out[205]: 29968.0

In [0]:
forecast.shape

Out[206]: (7292, 71)